In [62]:
using LinearAlgebra
using CSV
using DataFrames

function coordinate_descent(f, x0; step_size=0.01, tol=1e-6, max_iter=10000)
    x = copy(x0)
    n = length(x0)
    iter_count = 0
    iter_vectors = [vcat(x, f(x))]
    
    for iter in 1:max_iter
        x_old = copy(x)

        for i in 1:n
            f_current = f(x)
            x_forward = copy(x); x_forward[i] += step_size
            x_backward = copy(x); x_backward[i] -= step_size

            f_forward = f(x_forward)
            f_backward = f(x_backward)

            if f_forward < f_current && f_forward <= f_backward
                x[i] += step_size
            elseif f_backward < f_current
                x[i] -= step_size
            end
        end

        push!(iter_vectors, vcat(x, f(x)))
        
        if norm(x - x_old) < tol
            iter_count = iter
            break
        end
    end
    
    println("Число итераций метода покоординатного спуска: ", iter_count)
    return x, iter_vectors
end

# Функция обратного переменного шага для поиска оптимального α
function reverse_variable_step(f, x, e; Δ0=1.0, β=0.5, tol=1e-6, max_iter=10000)
    α0 = 0.0
    α1 = Δ0
    y0 = f(x + α0 * e)
    y1 = f(x + α1 * e)
    iter_count = 0

    for k in 1:max_iter
        if abs(y0 - y1) < tol
            iter_count = k
            break
        elseif y0 > y1
            α0, y0 = α1, y1
            α1 = α1 + Δ0
            y1 = f(x + α1 * e)
        else
            Δ0 *= -β
            α1 = α0 + Δ0
            y1 = f(x + α1 * e)
        end
    end
    return (α0 + α1) / 2
end

function gauss_seidel(f, x0; tol=1e-6, max_iter=10000)
    x = copy(x0)
    n = length(x)
    iter_count = 0
    iter_vectors = [vcat(x, f(x))]

    for k in 1:max_iter
        x_prev = copy(x)

        for i in 1:n
            e = zeros(n)
            e[i] = 1  # Единичный вектор вдоль i-й координаты
            
            # Оптимизируем по α
            α_opt = reverse_variable_step(f, x, e)
            x += α_opt * e
            push!(iter_vectors, vcat(x, f(x)))
        end

        if norm(f(x_prev) - f(x)) < tol
            iter_count = k
            break
        end
    end
    println("Число итераций метода Гаусса-Зейделя: ", iter_count)
    return x, iter_vectors
end

function hooke_jeeves(f, x0; Δ=0.1, β=0.5, tol=1e-6, max_iter=10000)
    x = copy(x0)
    n = length(x)
    Δx = fill(Δ, n)  # Вектор шагов по каждой координате
    iter_count = 0
    iter_vectors = [vcat(x, f(x))]

    while maximum(abs.(Δx)) > tol && iter_count < max_iter
        x_prev = copy(x)
        x_new = copy(x)
        a = 0

        # Исследующий поиск
        for i in 1:n
            x_test = copy(x_new)
            x_test[i] += Δx[i]
            if f(x_test) < f(x_new)
                x_new = x_test
                push!(iter_vectors, vcat(x_new, f(x_new)))
            else
                x_test[i] = x_new[i] - Δx[i]  # Пробуем в другую сторону
                if f(x_test) < f(x_new)
                    x_new = x_test
                    push!(iter_vectors, vcat(x_new, f(x_new)))
                end
            end
        end

        if x_new == x  # Если все шаги неудачны
            Δx *= β   # Уменьшаем шаги
        else
            # Движение по образцу
            x = 2 * x_new - x_prev
            if f(x) > f(x_new)  # Если движение по образцу не уменьшает f
                x = x_new  # Оставляем только исследующий поиск
            end
        end

        iter_count += 1
    end
    println("Число итераций метода Хука-Дживса: ", iter_count)
    return x, iter_vectors
end

function rosenbrock(x)
    return (1 - x[1])^2 + 100 * (x[2] - x[1]^2)^2
end

function schwefel(x1, x2)
    return 418.9829 * 2 - (x1 * sin(sqrt(abs(x1))) + x2 * sin(sqrt(abs(x2))))
end

function rastrigin(x)
    return 20 + sum(x.^2 .- 10 * cos.(2 * π * x))
end
function save_trajectory_to_file(traj, filename)
    CSV.write(filename, DataFrame(x1=[point[1] for point in traj], x2=[point[2] for point in traj], x3=[point[3] for point in traj]))
end

x0 = [0.4, -0.2]  
true_minimum = [1.0, 1.0] 

println("\n--- Покоординатный спуск для Розенброка ---")
@time xmin_cd, traj_cd = coordinate_descent(rosenbrock, x0)
println("Минимум найден в точке: ", xmin_cd)
println("Погрешность: ", norm(xmin_cd - true_minimum))

println("\n--- Гаусс-Зейдель для Розенброка ---")
@time xmin_gs, traj_gs = gauss_seidel(rosenbrock, x0)
println("Минимум найден в точке: ", xmin_gs)
println("Погрешность: ", norm(xmin_gs - true_minimum))

println("\n--- Хук-Дживс для Розенброка ---")
@time xmin_hj, traj_hj = hooke_jeeves(rosenbrock, x0)
println("Минимум найден в точке: ", xmin_hj)
println("Погрешность: ", norm(xmin_hj - true_minimum))

save_trajectory_to_file(traj_cd, "traj_cd_rosenbrock.csv")
save_trajectory_to_file(traj_gs, "traj_gs_rosenbrock.csv")
save_trajectory_to_file(traj_hj, "traj_hj_rosenbrock.csv")

println("\n--- Покоординатный спуск для Швефеля ---")
@time xmin_cd, traj_cd = coordinate_descent(schwefel, x0)
println("Минимум найден в точке: ", xmin_cd)

println("\n--- Гаусс-Зейдель для Швефеля ---")
@time xmin_gs, traj_gs = gauss_seidel(schwefel, x0)
println("Минимум найден в точке: ", xmin_gs)

println("\n--- Хук-Дживс для Швефеля ---")
@time xmin_hj, traj_hj = hooke_jeeves(schwefel, x0)
println("Минимум найден в точке: ", xmin_hj)

save_trajectory_to_file(traj_cd, "traj_cd_schwefel.csv")
save_trajectory_to_file(traj_gs, "traj_gs_schwefel.csv")
save_trajectory_to_file(traj_hj, "traj_hj_schwefel.csv")

println("\n--- Покоординатный спуск для Растригина ---")
@time xmin_cd, traj_cd = coordinate_descent(rastrigin, x0)
println("Минимум найден в точке: ", xmin_cd)

println("\n--- Гаусс-Зейдель для Растригина ---")
@time xmin_gs, traj_gs = gauss_seidel(rastrigin, x0)
println("Минимум найден в точке: ", xmin_gs)

println("\n--- Хук-Дживс для Растригина ---")
@time xmin_hj, traj_hj = hooke_jeeves(rastrigin, x0)
println("Минимум найден в точке: ", xmin_hj)

save_trajectory_to_file(traj_cd, "traj_cd_rastrigin.csv")
save_trajectory_to_file(traj_gs, "traj_gs_rastrigin.csv")
save_trajectory_to_file(traj_hj, "traj_hj_rastrigin.csv")


--- Покоординатный спуск для Розенброка ---
Число итераций метода покоординатного спуска: 43
  0.041032 seconds (20.12 k allocations: 1.339 MiB, 98.35% compilation time)
Минимум найден в точке: [0.38, 0.14]
Погрешность: 1.0601886624558856

--- Гаусс-Зейдель для Розенброка ---
Число итераций метода Гаусса-Зейделя: 954
  0.055885 seconds (103.20 k allocations: 7.693 MiB, 92.06% compilation time)
Минимум найден в точке: [0.9783843994140625, 0.95716552734375]
Погрешность: 0.047979435557640795

--- Хук-Дживс для Розенброка ---
Число итераций метода Хука-Дживса: 2381
  0.092574 seconds (45.95 k allocations: 3.283 MiB, 98.01% compilation time)
Минимум найден в точке: [0.9996734619140504, 0.9993469238280973]
Погрешность: 0.0007301613574291057

--- Покоординатный спуск для Швефеля ---
Число итераций метода покоординатного спуска: 545
  0.035923 seconds (16.16 k allocations: 1.168 MiB, 97.11% compilation time)
Минимум найден в точке: [5.239999999999933, 5.239999999999933]

--- Гаусс-Зейдель для

"traj_hj_rastrigin.csv"